# Índice

1. [Introdução](#intro)
2. [Estimação do movimento de pixels](#estim)
3. [Desenvolvimento do classificador](#class)
4. [Animação](#animação)
5. [Resultados](#app)
6. [Conclusões](#conclusao)
7. [Referências](#refs)


<a id="intro"></a>

#### 1. Introdução

Este projeto tem como objetivo o desenvolvimento de um algoritmo de estimação e classificação de movimento para integração em aplicação gráfica interactiva.
É pretendido a realização de uma animação gráfica de um objecto simples, através da estimação e classificação de movimentos de uma mão registados numa sequência de vídeo. A animação é realizada recorrendo ao Pygame. Este algoritmo de visão por computador está familiarizado com a biblioteca de funções OpenCV (Open Source Computer Vision) e permite a execução da aplicação em tempo real para interação pessoa-máquina.

In [1]:
import cv2
import numpy as np
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


<a id="estim"></a>

#### 2. Estimação do movimento de pixels

Para cada par de imagens consecutivas na sequência de vídeo, estimou-se o movimento de pixels usando a deteção e seguimento da mão.
Em primeiro lugar, detetou-se a região da mão, recorrendo à deteção do tom de pele com base no espaço de cor RG-normalizado, e, em segundo, fez-se correspondência de regiões entre imagens consecutivas.

In [2]:
import cv2
import numpy as np

def skinColorDetention(frame, largura, altura):
    luzDireta = True
    if(luzDireta):
        min_YCrCb = np.array([100, 140, 110], np.uint8)
        max_YCrCb = np.array([255, 170, 127], np.uint8)
    else:
        min_YCrCb = np.array([0, 140, 110], np.uint8)
        max_YCrCb = np.array([100, 170, 127], np.uint8)

    str_elem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    image_YCrCb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)
    skin_region = cv2.inRange(image_YCrCb, min_YCrCb, max_YCrCb)
    skin_region_morph = cv2.morphologyEx(skin_region, cv2.MORPH_CLOSE, str_elem)

    cv2.imshow('Skin Region Morphology',skin_region)
    
    contours, _ = cv2.findContours(skin_region_morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    min_circularity = float('inf')
    best_contour=None
    
    if contours:
        best_contour = max(contours, key=cv2.contourArea)

    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)

        if perimeter > 0: 
            circularity = (4 * np.pi * area) / (perimeter ** 2)

            if circularity < min_circularity and area > 0.02 * altura * largura:
                min_circularity = circularity
                best_contour = contour

    cv2.drawContours(frame, [best_contour], -1, (0, 255, 0), 2)

    if best_contour is not None and len(best_contour) >= 3:
        try:
            hull = cv2.convexHull(best_contour, returnPoints=False)
            defects = cv2.convexityDefects(best_contour, hull)

            if defects is not None:
                for i in range(defects.shape[0]):
                    _, _, far_idx, _ = defects[i, 0]
                    far = tuple(best_contour[far_idx][0])
                    cv2.circle(frame, far, 5, (0, 0, 255), -1)
            else:
                print("Nenhum defeito de convexidade encontrado.")
        except cv2.error as e:
            print("Erro no cálculo de convexidade:", e)
    else:
        print("Nenhum contorno válido para cálculo de casco convexo.")
       
    return best_contour

Para a estimação dos movimentos dos píxeis, definiu-se os limites da cor da pele no espaço de cor YCrCb, com valores mínimos e máximos da luminância, da componente de crominância vermelha e da componente de crominância azul.

É definido um elemento estruturante para operações morfológicas e converte-se o frame para o espaço de cor YCrCb

Com este frame, cria-se a máscara que deterá as regiões de pele e aplica-se a ela operações morfológicas para a melhorar.

Após encontrar os contornos nas regiões detectadas como pele, focou-se na detenção da mão apenas, ignorando outras regiões. Para tal, identificou-se o contorno com a menor circularidade através da seguinte fórmula:
$$\frac{4 \cdot \pi \cdot \text{área}}{\text{perímetro}^2}$$

Para a menor circularidade, determina-se o contorno da mão.

Adicionalmente, calcula-se ainda os defeitos de convexidade de forma a desenhar pontos associados aos cantos da mão.

Um casco convexo é gerado para o melhor contorno, representando uma envoltória que cobre a forma sem reentrâncias. Os defeitos de convexidade são identificados como pontos onde o contorno está mais distante do casco convexo.

Cada defeito é representado por um array de 4 valores: ponto inicial do defeito no contorno, ponto final do defeito no contorno, o ponto mais distante do casco convexo e a profundidade do defeito.

<a id="class"></a>

####  3. Desenvolvimento do classificador

Desenvolveu-se um classificador quqem tem como função distinguir os seguintes seis tipos de movimentos: DIREITA, ESQUERDA, CIMA, BAIXO, ZOOM IN, ZOOM OUT. 

In [3]:
movimentos_realizados = []

def classification(frame, contour, centroides, areas):
    movimento = "NENHUM"
    
    if contour is not None:
        area = cv2.contourArea(contour)
        if area > 10000 :
            areas.append(area)
        
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                centroides.append((cx, cy))

            if len(centroides) > 1:
                dx = centroides[-1][0] - centroides[0][0]
                dy = centroides[-1][1] - centroides[0][1]

                if abs(dx) > abs(dy):
                    if dx > 50:
                        movimento = "ESQUERDA"
                    elif dx < -50:
                        movimento = "DIREITA"
                else:
                    if dy > 50:
                        movimento = "BAIXO"
                    elif dy < -50:
                        movimento = "CIMA"

            if len(areas) > 10:
                delta_area = areas[-1] - areas[0]
                if delta_area > 37000:
                    movimento = "ZOOM IN"
                elif delta_area < -37000:
                    movimento = "ZOOM OUT"

        if len(centroides) > 20:
            centroides.pop(0)
        if len(areas) > 20:
            areas.pop(0)
            
        if movimento != "NENHUM":
            movimentos_realizados.append(movimento)

    cv2.putText(frame, f"Movimento: {movimento}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

Este método é responsável por detetar e classificar os movimentos da mão. Analisa-se as mudanças no centroide e na área do contorno de modo a identificar movimentos.

Definiu-se um movimento padrão que será atualizado sempre que um movimento for detetado.

Se forem detetados pelo menos dois centroides, verifica-se:
* o deslocamento horizontal e vertical, para movimentos CIMA, BAIXO, ESQUERDA e DIREITA
* o número de áreas detetadas, para os movimentos ZOOM IN e ZOOM OUT, com thresold de 37000 e -37000, respetivamente.

<a id="animação"></a>

####   4. Animação

In [4]:
def objectAnimation(frame, movimentos, posicao_inicial=(200, 200), tamanho_objeto=10):
    x, y = posicao_inicial

    for movimento in movimentos:
        if movimento == "DIREITA":
            x -= 10
        elif movimento == "ESQUERDA":
            x += 10
        elif movimento == "CIMA":
            y -= 10
        elif movimento == "BAIXO":
            y += 10
        elif movimento == "ZOOM IN":
            tamanho_objeto += 10
        elif movimento == "ZOOM OUT":
            tamanho_objeto = max(10, tamanho_objeto - 10)
            
    if(x<0): x=0
    elif(x>625): x=625
    if(y<0): y=0
    elif(y>430): y=430
    cv2.circle(frame, (x, y), tamanho_objeto, (255, 0, 0), -1)

Na animação do objeto, para cada movimento guardado no array, altera-se os valores x e y do objeto ou o seu tamanho de acordo com o tipo de movimento:
- <b>DIREITA</b>: a posição x diminui.
- <b>ESQUERDA</b>: a posição x aumenta.
- <b>CIMA</b>: a posição y diminui.
- <b>BAIXO</b>: a posição y aumenta.
- <b>ZOOM IN</b>: O tamanho do objeto aumenta em 10 pixels.
- <b>ZOOM OUT</b>: O tamanho do objeto diminui em 10 pixels (mas não pode ser menor que 10).

Adicionou-se ainda um trecho responsável pela limitação do objeto dentro dos limites da tela.

<a id="result"></a>

####   5. Resultados

In [5]:
def app(video_source):

    captura = cv2.VideoCapture(video_source)
    if not captura.isOpened():
        print("Erro ao acessar a câmera!")
        return

    largura = int(captura.get(cv2.CAP_PROP_FRAME_WIDTH))
    altura = int(captura.get(cv2.CAP_PROP_FRAME_HEIGHT))

    centroides = []
    areas = []

    while True:
        ret, frame = captura.read()
        if not ret:
            print("Erro ao capturar frame!")
            break

        contour = skinColorDetention(frame, largura, altura)

        classification(frame, contour, centroides, areas)
        
        objectAnimation(frame, movimentos_realizados)

        cv2.flip(frame, 0)
        cv2.imshow("Detecção de Mão com Esqueleto", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    captura.release()
    cv2.destroyAllWindows()

app(0)


Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para cálculo de casco convexo.
Nenhum contorno válido para 

<a class="anchor" id="conclusao"></a>

#### 6. Conclusão

Neste primeiro trabalho prático, foi desenvolvido um algoritmo de estimação e classificação de movimento para integração em aplicação gráfica interactiva.

Foram exploradas técnicas para detetar e seguir o movimento da mão, classificar os seu movimentos e animar um objeto gráfico com base nos movimentos detetados.

Foram utilizadas abordagens para segmentar a mão, como subtração de imagens e deteção de tom de pele. A classificação dos movimentos baseou-se na trajetória dos centroides, fruto da variação da área do contorno da mão. Por fim, os movimentos foram usados para controlar a animação de um objeto gráfico. 

Encontramos algumas dificuldades, entre elas a existência de objetos no fundo com tons de castanho, que são à partida selecionados como pele se não definirmos alguns limiares para tal seleção. Para além disso, a ambiguidade da luminosidade foi um problema para definir o intervalo da cor a verificar, por isso usamos o modelo de cor YCbCr já que um dos seus parâmetros é a intensidade, que é diretamente relacionada com a luminosidade.

<a id="refs"></a>

#### 7. Referências

* <a href="https://docs.opencv.org/4.x/d7/dbd/group__imgproc.html" target="_blank">OpenCV: Image Processing</a>
* <a href="https://2425moodle.isel.pt/course/view.php?id=8502" target="_blank">Acetatos de Processamento de Imagem e Visão, Moodle ISEL</a>
*  C. Gonzalez, Rafael; E. Woods, Richard. Digital Image Processing. 3ª edição